<a href="https://colab.research.google.com/github/jananimylswamy/designsprint-janani/blob/main/NLP_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
import json

with open("train-claims.json", "r") as f:
    train_claims = json.load(f)

with open("evidence.json", "r") as f:
    evidence_corpus = json.load(f)


In [2]:
import json

# Load the data
with open("train-claims.json", "r") as f:
    train_claims = json.load(f)

with open("evidence.json", "r") as f:
    evidence_corpus = json.load(f)

# Preview the first 5 entries
for i, (claim_id, claim_data) in enumerate(train_claims.items()):
    print(f"🔹 Claim ID: {claim_id}")
    print(f"   ✏️ Claim Text: {claim_data['claim_text']}")
    print(f"   🏷️ Label: {claim_data['claim_label']}")
    print(f"   📄 Evidence IDs: {claim_data['evidences']}")

    # Show the evidence texts
    for eid in claim_data["evidences"]:
        evidence_text = evidence_corpus.get(eid, "[MISSING]")
        print(f"      ➤ {eid}: {evidence_text}")

    print("-" * 80)

    if i == 4:
        break  # Show only first 5 claims


🔹 Claim ID: claim-1937
   ✏️ Claim Text: Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.
   🏷️ Label: DISPUTED
   📄 Evidence IDs: ['evidence-442946', 'evidence-1194317', 'evidence-12171']
      ➤ evidence-442946: At very high concentrations (100 times atmospheric concentration, or greater), carbon dioxide can be toxic to animal life, so raising the concentration to 10,000 ppm (1%) or higher for several hours will eliminate pests such as whiteflies and spider mites in a greenhouse.
      ➤ evidence-1194317: Plants can grow as much as 50 percent faster in concentrations of 1,000 ppm CO 2 when compared with ambient conditions, though this assumes no change in climate and no limitation on other nutrients.
      ➤ evidence-12171: Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.
-----------------------------------------------------------------

In [6]:
!pip uninstall -y nltk
!pip install nltk --upgrade --force-reinstall


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text, remove_stopwords=False):
    # Lowercase
    text = text.lower()

    # Remove punctuation and non-alphanumeric characters
    text = re.sub(r"[^a-z0-9\s]", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Tokenize
    tokens = nltk.word_tokenize(text)

    # Lemmatize and filter
    cleaned_tokens = []
    for token in tokens:
        if token.isalnum():  # keep only alphanumeric
            if len(token) > 1:  # remove single characters
                lemma = lemmatizer.lemmatize(token)
                if not remove_stopwords or lemma not in stop_words:
                    cleaned_tokens.append(lemma)

    return " ".join(cleaned_tokens)


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (792 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached joblib-1.5.0-py3-none-any.whl (307 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling r

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
nltk.download('punkt_tab')
for eid in evidence_corpus:
    evidence_corpus[eid] = clean_text(evidence_corpus[eid], remove_stopwords=False)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
positive_pairs = []

for cid, cdata in train_claims.items():
    claim = clean_text(cdata["claim_text"], remove_stopwords=False)
    for eid in cdata["evidences"]:
        if eid in evidence_corpus:
            evidence = evidence_corpus[eid]
            if len(evidence.split()) > 5:  # filter very short texts
                positive_pairs.append((claim, evidence, 1))


In [8]:
import random

negative_pairs = []
all_eids = set(evidence_corpus.keys())

for cid, cdata in train_claims.items():
    claim = clean_text(cdata["claim_text"], remove_stopwords=False)
    gold_eids = set(cdata["evidences"])
    candidate_neg_eids = list(all_eids - gold_eids)
    sampled_neg_eids = random.sample(candidate_neg_eids, k=3)  # 3 negatives per claim

    for eid in sampled_neg_eids:
        evidence = evidence_corpus[eid]
        if len(evidence.split()) > 5:
            negative_pairs.append((claim, evidence, 0))


In [9]:
dataset = positive_pairs + negative_pairs
random.shuffle(dataset)


In [10]:
for i in range(3):
    print(f"Claim: {dataset[i][0]}")
    print(f"Evidence: {dataset[i][1]}")
    print(f"Label: {dataset[i][2]}")
    print("---")


Claim: during the recent global warming summit in copenhagen nancy pelosi and others stayed at five star hotel on trip costing nearly 10 000 per person
Evidence: the document wa subtitled a the copenhagen agreement and proposes measure to keep average global temperature rise to two degree celsius above pre industrial level
Label: 1
---
Claim: and since the last ice age ended almost exactly 11 500 year ago ice age now
Evidence: the last cold episode of the last glacial period ended about 10 000 year ago
Label: 1
---
Claim: the modeler confused cause and effect thereby getting the feedback in the wrong direction
Evidence: the episode begin with the planet express ship crash landing on earth killing many of the main character
Label: 0
---


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

Step 1: Build a Vocabulary
You need to turn words into numbers (token IDs).


In [11]:
from collections import Counter

# Build vocab from all claim + evidence tokens
counter = Counter()

for claim, evidence, _ in dataset:
    counter.update(claim.split())
    counter.update(evidence.split())

# Special tokens
vocab = {"[PAD]": 0, "[CLS]": 1, "[SEP]": 2, "[UNK]": 3}
for word in counter:
    if word not in vocab:
        vocab[word] = len(vocab)


Step 2: Encode Text into Token IDs
Create input sequences like:

[CLS] claim tokens [SEP] evidence tokens [SEP]


In [12]:
MAX_LEN = 128

def encode(claim, evidence, vocab):
    tokens = ["[CLS]"] + claim.split() + ["[SEP]"] + evidence.split() + ["[SEP]"]
    token_ids = [vocab.get(tok, vocab["[UNK]"]) for tok in tokens]

    if len(token_ids) < MAX_LEN:
        token_ids += [vocab["[PAD]"]] * (MAX_LEN - len(token_ids))
    else:
        token_ids = token_ids[:MAX_LEN]

    return token_ids


Now encode all your data:


In [13]:
input_ids = []
labels = []

for claim, evidence, label in dataset:
    encoded = encode(claim, evidence, vocab)
    input_ids.append(encoded)
    labels.append(label)


Step 3: Create a PyTorch Dataset
This wraps your input data for training:

In [14]:
import torch
from torch.utils.data import Dataset

class EvidenceDataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx]), torch.tensor(self.labels[idx], dtype=torch.float)


Create the dataset and dataloader:


In [15]:
from torch.utils.data import DataLoader

dataset = EvidenceDataset(input_ids, labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


Step 4: Define a Simple Transformer Model
This model reads your token IDs and outputs a score (0–1) for each pair:

In [16]:
import torch.nn as nn

class ShallowTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, max_len=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=4, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids):
        x = self.embedding(input_ids) + self.pos_embedding[:, :input_ids.size(1), :]
        x = self.transformer(x)
        cls_token = x[:, 0, :]  # Use [CLS] token output
        return self.classifier(cls_token).squeeze(-1)  # shape: [batch_size]


Step 5: Train the Model


In [17]:
model = ShallowTransformer(vocab_size=len(vocab))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()


In [18]:
for epoch in range(5):
    total_loss = 0
    model.train()

    for batch_inputs, batch_labels in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_inputs)
        loss = criterion(predictions, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


Epoch 1, Loss: 0.6516
Epoch 2, Loss: 0.5681
Epoch 3, Loss: 0.4878
Epoch 4, Loss: 0.4305
Epoch 5, Loss: 0.3886


Evaluate Model Performance


In [20]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for input_ids, labels in train_loader:  # use validation/test data
        outputs = model(input_ids)
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {correct / total:.2%}")


Accuracy: 86.70%


Validation Set

In [25]:
# Step 1: Load dev-claims.json
with open("dev-claims.json") as f:
    dev_claims = json.load(f)

# Step 2: Create positive/negative (claim, evidence, label) pairs
val_dataset = []
for cid, data in dev_claims.items():
    claim_text = clean_text(data["claim_text"])
    gold_evidence_ids = set(data["evidences"])

    # Positive examples
    for eid in gold_evidence_ids:
        if eid in evidence_corpus:
            evidence = clean_text(evidence_corpus[eid])
            val_dataset.append((claim_text, evidence, 1))

    # Negative examples (sample a few that are not gold)
    negative_ids = list(set(evidence_corpus.keys()) - gold_evidence_ids)
    sampled = random.sample(negative_ids, 3)
    for eid in sampled:
        evidence = clean_text(evidence_corpus[eid])
        val_dataset.append((claim_text, evidence, 0))

# Step 3: Encode
val_input_ids = [encode(c, e, vocab) for c, e, _ in val_dataset]
val_labels = [lbl for _, _, lbl in val_dataset]

# Step 4: Wrap into DataLoader
val_loader = DataLoader(EvidenceDataset(val_input_ids, val_labels), batch_size=32)


In [26]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for input_ids, labels in val_loader:  # ✅ use val_loader here
        outputs = model(input_ids)
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {correct / total:.2%}")


Validation Accuracy: 80.27%


In [21]:
torch.save(model.state_dict(), "shallow_transformer.pth")

In [27]:
import torch
import heapq  # for efficient top-k

def retrieve_top_k_evidence(claim, evidence_corpus, model, vocab, k=5, max_len=128):
    model.eval()
    scored_passages = []

    with torch.no_grad():
        for eid, evidence in evidence_corpus.items():
            input_ids = encode(claim, evidence, vocab)
            input_tensor = torch.tensor([input_ids])  # batch size = 1

            score = model(input_tensor).item()  # relevance score (0 to 1)
            scored_passages.append((score, eid, evidence))

    # Sort by score descending and return top-k
    top_k = heapq.nlargest(k, scored_passages, key=lambda x: x[0])

    return top_k


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*